In [31]:
import csv
import pandas as pd
import glob
import numpy as np
import geopandas
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
life_exp_us = pd.read_excel('data/IHME/ihme.xlsx', header=1)
life_exp_us['FIPS'] = life_exp_us['FIPS'].astype(dtype=pd.Int64Dtype())


In [33]:
life_exp_us['FIPS'] = life_exp_us['FIPS'].apply(lambda x: str(x).ljust(4, '0') if(np.all(pd.notnull(x))) else x)

In [35]:
life_exp_us.head()

,Location,FIPS,"Life expectancy, 1980*","Life expectancy, 1985*","Life expectancy, 1990*","Life expectancy, 1995*","Life expectancy, 2000*","Life expectancy, 2005*","Life expectancy, 2010*","Life expectancy, 2014*","% Change in Life Expectancy, 1980-2014"
0,United States,NaN,"73.75 (73.73, 73.77)","74.72 (74.71, 74.74)","75.39 (75.38, 75.41)","75.86 (75.85, 75.88)","76.94 (76.92, 76.95)","77.65 (77.61, 77.68)","78.82 (78.81, 78.84)","79.08 (79.04, 79.11)","7.22 (7.16, 7.27)"
1,Alabama,1000,"72.29 (72.21, 72.37)","73.10 (73.02, 73.19)","73.50 (73.42, 73.57)","73.78 (73.70, 73.85)","74.46 (74.39, 74.53)","74.71 (74.62, 74.79)","75.58 (75.51, 75.65)","75.65 (75.57, 75.72)","4.65 (4.48, 4.82)"
2,"Autauga County, Alabama",1001,"71.99 (71.33, 72.63)","72.70 (72.10, 73.32)","73.10 (72.54, 73.66)","73.69 (73.22, 74.20)","74.55 (74.10, 75.04)","74.71 (74.23, 75.20)","75.74 (75.25, 76.25)","75.67 (75.15, 76.18)","5.10 (3.86, 6.43)"
3,"Baldwin County, Alabama",1003,"74.07 (73.60, 74.58)","74.92 (74.48, 75.37)","75.31 (74.88, 75.71)","75.79 (75.40, 76.20)","76.80 (76.45, 77.17)","77.09 (76.72, 77.47)","77.80 (77.45, 78.14)","78.08 (77.72, 78.43)","5.41 (4.51, 6.26)"
4,"Barbour County, Alabama",1005,"71.52 (70.79, 72.20)","72.48 (71.92, 73.10)","72.80 (72.23, 73.40)","73.24 (72.69, 73.81)","74.13 (73.59, 74.64)","74.30 (73.69, 74.87)","75.34 (74.72, 75.95)","75.42 (74.78, 76.07)","5.46 (4.10, 6.90)"


In [36]:
life_exp_counties = life_exp_us[life_exp_us['Location'].str.contains('County')]
life_exp_counties.head()

,Location,FIPS,"Life expectancy, 1980*","Life expectancy, 1985*","Life expectancy, 1990*","Life expectancy, 1995*","Life expectancy, 2000*","Life expectancy, 2005*","Life expectancy, 2010*","Life expectancy, 2014*","% Change in Life Expectancy, 1980-2014"
2,"Autauga County, Alabama",1001,"71.99 (71.33, 72.63)","72.70 (72.10, 73.32)","73.10 (72.54, 73.66)","73.69 (73.22, 74.20)","74.55 (74.10, 75.04)","74.71 (74.23, 75.20)","75.74 (75.25, 76.25)","75.67 (75.15, 76.18)","5.10 (3.86, 6.43)"
3,"Baldwin County, Alabama",1003,"74.07 (73.60, 74.58)","74.92 (74.48, 75.37)","75.31 (74.88, 75.71)","75.79 (75.40, 76.20)","76.80 (76.45, 77.17)","77.09 (76.72, 77.47)","77.80 (77.45, 78.14)","78.08 (77.72, 78.43)","5.41 (4.51, 6.26)"
4,"Barbour County, Alabama",1005,"71.52 (70.79, 72.20)","72.48 (71.92, 73.10)","72.80 (72.23, 73.40)","73.24 (72.69, 73.81)","74.13 (73.59, 74.64)","74.30 (73.69, 74.87)","75.34 (74.72, 75.95)","75.42 (74.78, 76.07)","5.46 (4.10, 6.90)"
5,"Bibb County, Alabama",1007,"71.61 (70.86, 72.36)","72.17 (71.48, 72.83)","72.61 (72.01, 73.22)","72.62 (72.02, 73.16)","73.56 (72.98, 74.14)","73.38 (72.78, 73.97)","74.13 (73.48, 74.77)","73.97 (73.28, 74.68)","3.29 (1.83, 4.92)"
6,"Blount County, Alabama",1009,"74.01 (73.40, 74.64)","74.50 (73.97, 75.05)","74.70 (74.19, 75.21)","74.80 (74.29, 75.29)","75.62 (75.16, 76.05)","75.71 (75.25, 76.19)","76.41 (75.96, 76.91)","76.16 (75.66, 76.69)","2.91 (1.76, 4.10)"


In [ ]:
life_exp_counties = 
w['female'] = w['female'].map({'female': 1, 'male': 0})

In [78]:
counties = geopandas.read_file('data/TIGER/cb_2015_us_county_500k')
states = geopandas.read_file('data/TIGER/cb_2015_us_state_500k')

In [48]:
life_exp_counties[life_exp_counties['Location']=='Summit County, Colorado']['Life expectancy, 1980*'].str[:5]

310    79.18
Name: Life expectancy, 1980*, dtype: object

In [52]:
life_exp_counties.filter(like='*').columns

Index(['Life expectancy, 1980*', 'Life expectancy, 1985*',
       'Life expectancy, 1990*', 'Life expectancy, 1995*',
       'Life expectancy, 2000*', 'Life expectancy, 2005*',
       'Life expectancy, 2010*', 'Life expectancy, 2014*'],
      dtype='object')

In [74]:
df=life_exp_counties.filter(like='*').columns
for col in df:
    life_exp_counties[col] = life_exp_counties[col].map(lambda x: float(x.partition(' ')[0]))

/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [75]:
life_exp_counties.head()

,Location,FIPS,"Life expectancy, 1980*","Life expectancy, 1985*","Life expectancy, 1990*","Life expectancy, 1995*","Life expectancy, 2000*","Life expectancy, 2005*","Life expectancy, 2010*","Life expectancy, 2014*","% Change in Life Expectancy, 1980-2014"
2,"Autauga County, Alabama",1001,71.99,72.70,73.10,73.69,74.55,74.71,75.74,75.67,"5.10 (3.86, 6.43)"
3,"Baldwin County, Alabama",1003,74.07,74.92,75.31,75.79,76.80,77.09,77.80,78.08,"5.41 (4.51, 6.26)"
4,"Barbour County, Alabama",1005,71.52,72.48,72.80,73.24,74.13,74.30,75.34,75.42,"5.46 (4.10, 6.90)"
5,"Bibb County, Alabama",1007,71.61,72.17,72.61,72.62,73.56,73.38,74.13,73.97,"3.29 (1.83, 4.92)"
6,"Blount County, Alabama",1009,74.01,74.50,74.70,74.80,75.62,75.71,76.41,76.16,"2.91 (1.76, 4.10)"


In [79]:
states.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,31,01779792,0400000US31,31,NE,Nebraska,00,198972429772,1356294775,"POLYGON ((-104.053418545071 41.1705389679833, ..."
1,53,01779804,0400000US53,53,WA,Washington,00,172121026344,12540093192,"(POLYGON ((-122.33164 48.020556, -122.328343 4..."
2,35,00897535,0400000US35,35,NM,New Mexico,00,314161426332,755674004,"POLYGON ((-109.050173 31.480004, -109.049843 3..."
3,46,01785534,0400000US46,46,SD,South Dakota,00,196349394266,3379810671,"POLYGON ((-104.057698 44.997431, -104.049505 4..."
4,21,01779786,0400000US21,21,KY,Kentucky,00,102266253024,2389483091,"(POLYGON ((-89.405654 36.528165, -89.398685 36..."
